# Utils 
This page shows the inside of the utils.py script, which allows for common functions to be easily used across different notebooks in the book without redefining them each time.

## Import notebook dependencies

In [1]:
import os
import numpy as np
import pandas as pd 
import xarray as xr
from glob import glob
import sys
import scipy
import matplotlib.pyplot as plt
import numpy.ma as ma
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pyproj
from datetime import date
from matplotlib.axes import Axes
from cartopy.mpl.geoaxes import GeoAxes
GeoAxes._pcolormesh_patched = Axes.pcolormesh

## Functions

### getWinterDateRange
Gets date range for winter season/s

In [2]:
def getWinterDateRange(start_year, end_year): 
    """ Gets date range for winter season/s
    Args: 
        start_year (int): start year 
        end_year (int): end year 
        
    Returns: 
        winters (list): list of dates for all winter seasons in the input range (i.e: ['1980-11','1980-12','1981-01',
         '1981-02','1981-03','1981-04')
    """
    winters = []
    for year in range(start_year, end_year, 1):
        winters += pd.date_range(start = str(year) + '-11', end = str(year + 1) + '-04', freq = 'MS')
    return winters

### getIS2Data
Gets ICESat-2 data for provided date range

In [3]:
def getIS2Data(dataPath, dates): 
    """Gets ICESat-2 data for provided date range
    
    Args: 
        dataPath (str): path to local directory of ICESat-2 data 
        dates (list): pandas Timestamp objects generated by getWinterDateRange
        
    Returns: 
        is2 (xarray dataset): ICESat-2 data or NONE if file does not exist for inputted date range
    """
    is2List = [] #empty list for compiling xarray DataArray objects
    for date in dates: 
        try:
            filename = glob(dataPath+ 'IS2*' + date.strftime('%y') + date.strftime('%m') + '*.nc')[0]
        except: 
            print('Cannot find files; check date range or filepath')
            return None
        is2 = xr.open_dataset(filename)
        is2 = is2.assign_coords({'time': date})
        is2List.append(is2)
    is2Data = xr.concat(is2List, dim = 'time') #concatenate all DataArray objects into a single DataArray
    return is2Data

### getSICData
Gets sea ice concentration data for provided date range

In [4]:
def getSICData(dataPath, dates): 
    """Gets sea ice concentration data for provided date range
    
    Args: 
        dataPath (str): path to local directory of sea ice concentration data 
        dates (list): pandas Timestamp objects generated by getWinterDateRange
        
    Returns: 
        sicData (xarray dataset): sea ice concentration data or NONE if file does not exist for inputted date range
    """
    sicList = [] #empty list for compiling xarray DataArray objects
    for date in dates: 
        try:
            filename = glob(dataPath+ 'seaice_conc_monthly*' + date.strftime('%Y') + date.strftime('%m') + '*.nc')[0]
        except: 
            print('Cannot find files; check date range, filepath or if glob is imported')
            return None
        dropVariables = ['goddard_merged_seaice_conc_monthly','goddard_bt_seaice_conc_monthly', 'goddard_nt_seaice_conc_monthly']
        sicList.append(xr.open_dataset(filename, drop_variables = dropVariables))
    sicData = xr.concat(sicList , dim = 'time', combine_attrs = 'override', data_vars = 'minimal') #concatenate all DataArray objects into a single DataArray
    return sicData

### getRegionMask
Gets NSIDC region mask for map projection

In [5]:
def getRegionMask(dataPath): 
    """Gets NSIDC region mask for map projection
    
     Args:
         dataPath (str): path to NSIDC region mask
         
     Returns: 
         shapedMask (numpy array): NSIDC arctic region mask gridded to shape [448, 304]
         shapedLons (numpy array): longitudes gridded to shape [448, 304]
         shapedLats (numpy array): latitudes gridded to shape [448, 304] 
    """ 
    gridShape = [448, 304] #shape of grid to reshape data to 
    
    regionMask = open(dataPath + '/sect_fixed_n.msk', 'rb') #open region mask 
    shapedMask = np.reshape(np.fromfile(file = regionMask, dtype='uint8'), gridShape) #reshape mask to grid shape
    
    maskLons = open(dataPath + '/psn25lons_v3.dat', 'rb') #open region mask longitudes
    maskLats = open(dataPath + '/psn25lats_v3.dat', 'rb') #open region mask latitudes
    shapedLons = np.reshape(np.fromfile(file = maskLons, dtype='<i4')/100000., gridShape) #reshape longitudes to grid shape
    shapedLats = np.reshape(np.fromfile(file = maskLats, dtype='<i4')/100000., gridShape) #reshape latitudes to grid shape

    return shapedMask, shapedLons, shapedLats

### getPIOMASData
Gets PIOMAS mean monthly sea ice thickness data for an input time range

In [ ]:
def getPIOMASData(dataPath, startYear = 1978, endYear = 2020): 
    """Gets PIOMAS mean monthly sea ice thickness data for an input time range. 
    
    Args: 
        dataPath (str): path to local directory of PIOMAS data 
        startYear (int): year to start loading data from (defaul to 1978)
        endYear (int): year to stop loading data from (default to 2020)
    
    Returns: 
        PIOMAS_data (xarray DataArray): PIOMAS data with descriptive coordinates and attributes
        
    Note: last available PIOMAS data was from July 2020 at the time of creation of this function
    """
    dataList = [] #empty list for compiling data

    for year in range(startYear, endYear + 1, 1):     
        data = open(dataPath + '/heff.H' + str(year), 'rb') 
        if year == 2020: #need special reshaping for 2020 because we dont have a full year of data
            period = 7
        else:
            period = 12
        dataList += list(np.fromfile(file = data, dtype='f').reshape([period, 120, 360]))
    
    #add latitude and longitude 
    gridP = np.loadtxt(dataPath + 'grid.dat.txt')
    lonsP = gridP[0:4320, :].flatten()
    lonsP = np.reshape(lonsP, [120,360])
    latsP = gridP[4320:, :].flatten()
    latsP = np.reshape(latsP, [120,360])
    
    #load dataList as an xarray DataArray with descriptive attributes and coordinates
    time = pd.date_range(start = str(startYear), end = str(endYear) + '-07', freq = 'MS')
    PIOMAS_attrs = {'units': 'meters', 'long_name': 'PIOMAS sea ice thickness', 'data description': 'PIOMAS monthly mean sea ice thickness', 'citation': 'Zhang, J.L. and D.A. Rothrock, “Modeling global sea ice with a thickness and enthalpy distribution model in generalized curvilinear coordinates“, Mon. Weather Rev., 131, 845-861, 2003'}
    PIOMAS_data = xr.DataArray(dataList, dims = ['time','x','y'], coords = {'time': time, 'longitude': (('x','y'), lonsP), 'latitude': (('x','y'), latsP)}, attrs = PIOMAS_attrs)
    
    return PIOMAS_data

### getDriftData
Gets weekly NSIDC sea ice drift data for the Arctic, resamples monthly, and returns a dataset

In [ ]:
def getDriftData(dataPath):
    """Gets weekly NSIDC sea ice drift data for the Arctic, resamples monthly, and returns a dataset
    
    Args: 
        dataPath (str): path to local directory of weekly drift data 

    Returns: 
        monthlyDrifts (xarray DataArray): sea ice drift data, resampled to monthly means, with descriptive coordinates and attributes
        
    Note: This function uses two different datasets from the NSIDC for drift in order to get the most recent data. Drift dataset from 1978-2018 was combined with quicklook drift dataset from 2019-present, maintaining attitributes of the 1978-2018 dataset
    """
    def get_uv_from_xy(xdrift, ydrift, lon):
        """convert the drift vectors to zonal/meridional
        """
        alpha = lon*np.pi/180. #convert longitudes to radians 
        uvelT = ydrift*np.sin(alpha) + xdrift*np.cos(alpha)
        vvelT = ydrift*np.cos(alpha) - xdrift*np.sin(alpha) 
        return uvelT, vvelT
    
    #combine nc weekly files into a single xarray dataset
    files = [xr.open_dataset(dataPath + f) for f in os.listdir(dataPath) if os.path.isfile(dataPath + f) and f.endswith('.nc')]
    weeklyDrifts = xr.concat(files, dim = 'time')

    #get transformed u,v variables and add to dataset
    uvelT, vvelT = get_uv_from_xy(weeklyDrifts.u, weeklyDrifts.v, weeklyDrifts.longitude)
    weeklyDrifts = weeklyDrifts.assign(drifts_uT = uvelT, drifts_vT = vvelT)
    weeklyDrifts.drifts_uT.attrs = {'description':'along-x component of the ice motion (u variable) converted to zonal/meridional'}
    weeklyDrifts.drifts_vT.attrs = {'description':'along-y component of the ice motion (v variable) converted to zonal/meridional'}

    #resample to get monthly data 
    monthlyDrifts = weeklyDrifts.resample(time='MS', keep_attrs = True).mean()

    #convert to same time format as ICESat-2 
    monthlyDrifts = monthlyDrifts.assign_coords(time = [pd.to_datetime(date.strftime('%m-%d-%Y')) for date in monthlyDrifts.time.values])
    
    #add attributes 
    for var in monthlyDrifts.data_vars: 
        monthlyDrifts[var].attrs = weeklyDrifts[var].attrs
        if var == 'drifts_uT' or var == 'drifts_vT': 
            monthlyDrifts[var].attrs['long_name'] = 'NSIDC sea ice motion vectors'
            monthlyDrifts[var].attrs['units'] = 'cm/s'
    monthlyDrifts.attrs['njkeeney comment'] = 'drift dataset from 1978-2018 was combined with quicklook drift dataset from 2019-present, maintaining attitributes of the 1978-2018 dataset'
    monthlyDrifts.attrs['citation'] = 'Tschudi, M., W. N. Meier, J. S. Stewart, C. Fowler, and J. Maslanik. 2019. Polar Pathfinder Daily 25 km EASE-Grid Sea Ice Motion Vectors, Version 4. Weekly sea ice motion. Boulder, Colorado USA. NASA National Snow and Ice Data Center Distributed Active Archive Center. doi: https://doi.org/10.5067/INAWUWO7QH7B. July 2020.'
    
    return monthlyDrifts

### plotOneMonth
Plots map of the arctic on North Pole Stereo projection with one month of data overlayed, along with the sea ice edge for each month

In [ ]:
def plotOneMonth(dataset, dataVar, month, minval, maxval, cbarTicks = None, cmap = 'viridis'): 
    """Plots map of the arctic on North Pole Stereo projection with one month of data overlayed, along with the sea ice edge for each month.
   
    Args:
        dataset (xr Dataset): dataset from google bucket
        dataVar (str): variable of interest
        month (str): month and year of interest, i.e. 'Dec 2019' (does not need to be in any particular format)
        minval, maxval (int): minimum and maximum values for the data variable 
        cbarTicks (list or np array of length 2): ticks to use on colorbar (default to [minval + 1, maxval +1])
        cmap (str, optional): color map (default to viridis)
        
    Returns:
        Figure displayed in notebook 
    
    """
    
    #define projection and transform
    proj = ccrs.NorthPolarStereo(central_longitude = -45)
    transform = ccrs.PlateCarree()
    
    #initialize the figure and axes 
    plt.figure(figsize=(6, 6))
    ax = plt.axes(projection = proj)
    
    #define arguments if not inputted 
    cbarTicks = np.arange(minval, maxval + 1, 1) if cbarTicks is None else cbarTicks
    
    #plot sea ice concentraion 
    SICarray = dataset['seaice_conc_monthly_cdr'].sel(time = month).where(dataset['region_mask']!=21) #dont plot contour along coastlines
    
    #stackexchange workaround for plotting on a rotated grid
    lonGreater = ma.masked_greater(SICarray.longitude.values, -0.01)
    lonLesser = ma.masked_less(SICarray.longitude.values, 0)
    latGreater = ma.MaskedArray(SICarray.latitude.values, mask = lonGreater.mask)
    latLesser = ma.MaskedArray(SICarray.latitude.values, mask = lonLesser.mask)
    dataGreater = ma.MaskedArray(SICarray.values[0], mask = lonGreater.mask)
    dataLesser = ma.MaskedArray(SICarray.values[0], mask = lonLesser.mask)
    
    #plot contour using each part of the 2 masked data sets
    im2a = ax.contour(lonGreater, latGreater, dataGreater, levels = [0.5], transform = transform, colors = 'magenta', linewidths = 0.9, zorder=5, alpha=1)
    im2b = ax.contour(lonLesser, latLesser, dataLesser, levels = [0.5], transform = transform, colors = 'magenta', linewidths = 0.9, zorder=5, alpha=1)
    #im = ax.contour(SICarray.longitude.values, SICarray.latitude.values, SICarray.values[0], levels = [0.15], transform = transform, colors = 'magenta', linewidths = 0.8, zorder=15, alpha=1)
    
    #plot the data
    dataset[dataVar].where(dataset['seaice_conc_monthly_cdr'] > 0.5).sel(time = month).plot(x = 'longitude', y = 'latitude', vmin = minval, vmax = maxval, extend = 'both', 
                    ax = ax, add_colorbar = True, transform = transform, zorder = 2, cmap = cmap, 
                    cbar_kwargs = {'label': dataset[dataVar].attrs['long_name'] + ' (' + dataset[dataVar].attrs['units'] + ')', 'orientation': 'horizontal', 'shrink': 0.75, 'pad': 0.025})
    
    #add features to the map
    ax.coastlines(linewidth=0.15, color = 'black', zorder = 10) #add coastlines 
    ax.add_feature(cfeature.LAND, color ='0.95', zorder = 5) #add land 
    ax.add_feature(cfeature.LAKES, color = 'grey', zorder = 5) #add lakes 
    ax.gridlines(draw_labels = False, linewidth = 0.25, color = 'gray', alpha = 0.7, linestyle = '--', zorder = 6) #add gridlines
    ax.set_extent([-179, 179, 55, 90], crs = transform) #zoom in so map only displays the Arctic
    ax.set_title(month + ": " + dataset[dataVar].attrs['long_name'], fontweight = 'medium')
    
    #display figure in notebook 
    plt.show()

### plotWinterMaps
Plot maps of the arctic on North Pole Stereo projection with several months of data overlayed, along with the sea ice edge for each month. 

In [6]:
def plotWinterMaps(dataset, dataVar, winter, minval, maxval, cbarTicks = None, title = "", cmap = 'viridis'):
    """Plot maps of the arctic on North Pole Stereo projection with several months of data overlayed, along with the sea ice edge for each month. 
   
    Args:
        dataset (xr Dataset): dataset from google bucket 
        dataVar (str): variable of interest
        winter (list): list of pandas Timestamp objects generated by getWinterDateRange(startYear, endYear)
        minval, maxval (int): minimum and maximum values for the data variable 
        cbarTicks (list or np array of length 2): ticks to use on colorbar (default to [minval + 1, maxval +1])
        title (str, optional): title of subplots (default to empty string)
        cmap (str, optional): color map (default to viridis)

    Returns:
        Figure displayed in notebook 

    """
    #format time for plotting 
    timeFormatted = list(pd.to_datetime(winter).strftime('%B %Y'))
    
    #define projection and transform
    proj = ccrs.NorthPolarStereo(central_longitude = -45)
    transform = ccrs.PlateCarree()
    
    #define arguments if not inputted 
    cbarTicks = np.arange(minval, maxval + 1, 1) if cbarTicks is None else cbarTicks
    
    #plot the data
    im = dataset[dataVar].where(dataset['seaice_conc_monthly_cdr'] > 0.5).sel(time = winter).plot.pcolormesh(x = 'longitude', y = 'latitude', vmin = minval, vmax = maxval, cmap = cmap,
        extend='both', levels=20, transform = transform, col='time', add_colorbar = True, zorder = 2, figsize = (8,8), col_wrap = 3,
        cbar_kwargs = {'ticks': cbarTicks, 'label': dataset[dataVar].attrs['long_name'] + ' (' + dataset[dataVar].attrs['units'] + ')', 'orientation': 'horizontal', 'shrink': 0.4, 'pad': 0.03},
        subplot_kws = {'projection': proj})
    
    #add a title
    plt.suptitle(title + ': ' + dataset[dataVar].attrs['long_name'], fontsize = 20, y = 0.99, fontweight = 'medium')

    i = 0 #indexer to go through timeFormatted and winter arrays and assign the correct data to each month
    for ax in im.axes.flat:
        ax.coastlines(linewidth=0.25, color = 'black', zorder = 10) #add coastlines 
        ax.add_feature(cfeature.LAND, color ='0.95', zorder = 5) #add land 
        ax.add_feature(cfeature.LAKES, color = 'grey', zorder = 5) #add lakes 
        ax.gridlines(draw_labels = False, linewidth = 0.25, color = 'gray', alpha = 0.75, linestyle='--', zorder = 6) #add gridlines
        ax.set_extent([-179, 179, 50, 90], crs = transform) #zoom in so map only displays the Arctic

        #plot sea ice concentration 
        SICarray = dataset['seaice_conc_monthly_cdr'].sel(time = winter[i]).where(dataset['region_mask']!=21) #dont plot contour along coastlines
        lonGreater = ma.masked_greater(SICarray.longitude.values, -0.01)
        lonLesser = ma.masked_less(SICarray.longitude.values, 0)
        latGreater = ma.MaskedArray(SICarray.latitude.values, mask = lonGreater.mask)
        latLesser = ma.MaskedArray(SICarray.latitude.values, mask = lonLesser.mask)
        dataGreater = ma.MaskedArray(SICarray.values, mask = lonGreater.mask)
        dataLesser = ma.MaskedArray(SICarray.values, mask = lonLesser.mask)
        im2a = ax.contour(lonGreater, latGreater, dataGreater, levels = [0.5], transform = transform, colors = 'magenta', linewidths = 0.8, zorder=5, alpha=1)
        im2b = ax.contour(lonLesser, latLesser, dataLesser, levels = [0.5], transform = transform, colors = 'magenta', linewidths = 0.8, zorder=5, alpha=1)
    
        #set title of each plot to formatted date 
        ax.set_title(timeFormatted[i])
        
        #update indexer 
        i += 1
    
    #display figure in notebook 
    plt.show()

### restrictRegionally
Restrict dataset to input regions

In [7]:
def restrictRegionally(dataset, regionKeyList): 
    """Restrict dataset to input regions.
    
    Args: 
        dataset (xr Dataset): dataset generated by Load_IS2 notebook
        regionKeyList (list): list of region keys to restrict data to 
        
    Returns: 
        regionalDataset (xr Dataset): dataset with restricted data to input regions
    """
    
    def checkKeys(regionKeyList, regionTbl): 
        """Check that regionKeyList was defined correctly

        Raises: 
            ValueError if regionKeyList was not defined correctly 
            warning if all data was removed from the dataset
        """
        if type(regionKeyList) != list: #raise a ValueError if regionKeyList is not a list 
            raise ValueError('regionKeyList needs to be a list. \nFor example, if you want to restrict data to the Beaufort Sea, define regionKeyList = [13]')

        for key in regionKeyList: 
            if key not in list(regionTbl['key']): 
                raise ValueError('Region key ' + str(key) + ' does not exist in region mask. \n Redefine regionKeyList with key numbers from table')

        if len(regionKeyList) == 0: 
            warnings.warn('You removed all the data from the dataset. Are you sure you wanted to do this? \n If not, make sure the list regionKeyList is not empty and try again. \n If you intended to keep data from all regions, set regionKeyList = list(tbl[\"key\"])')
 
    #create a table of keys and labels
    regionMask = dataset.region_mask.attrs
    regionTbl = pd.DataFrame({'key': regionMask['keys'], 'label': regionMask['labels']})
    
    #call function to check if regionKeyList was defined correctly
    checkKeys(regionKeyList, regionTbl)
    
    #keys to remove (all keys that are note listed in regionKeyList)
    keysToRemove = [key for key in list(regionTbl['key']) if key not in regionKeyList]
    
    #filter elements from the ice thickness DataArray where the region is the desired region
    regionalDataset = dataset.copy()
    for var in dataset.data_vars: 
        if var != 'seaice_conc_monthly_cdr':
            regionalVar = regionalDataset[var]
            for key in keysToRemove: 
                regionalVar = regionalVar.where(regionalVar['region_mask'] != key)
            regionalDataset[var] = regionalVar
    
    #find name of labels 
    labels = [regionTbl[regionTbl['key'] == key]['label'].item() for key in regionKeyList]
    
    #add new attributes describing changes made to the dataset
    if len(labels) < len(regionTbl['key']): 
        if set(regionKeyList) == set([10,11,12,13,15]): #convert to sets so unordered lists are compared
            regionalDataset.attrs['regions with data'] = 'Inner Arctic'
        else:    
            regionalDataset.attrs['regions with data'] = ('%s' % ', '.join(map(str, labels)))
        print('Regions selected: ' + regionalDataset.attrs['regions with data'])
    else: 
        regionalDataset.attrs['regions with data'] = 'All'
        print('Regions selected: All \nNo regions will be removed')
    
    return regionalDataset

### progressBar
Display a progress bar inside a for loop. Used in code development.

In [8]:
def progressBar(i, tot): 
    """
    Function: display a progress bar inside a for loop 
    Input: 
        -i: iteration number (integer value)
        -tot: total number of iterations (integer value)
    """
    j = (i + 1) / tot
    sys.stdout.write('\r [%-20s] %d%% complete' % ('='*int(20*j), 100*j))
    sys.stdout.flush()